In [ ]:
import requests_cache
import requests
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
import numpy as np
import seaborn as sns
from requests_oauthlib import OAuth1Session
import glob
import json
import os
import urllib2
from urllib2 import Request, urlopen
from requests_oauthlib import OAuth1Session
from bs4 import BeautifulSoup

### Data Wrangling Part 2 (From yelp dataset challenge):

In [ ]:
path = "/Users/Stephanie/Desktop/yelp_dataset_challenge_round9"
files = glob.glob(os.path.join(path,"*.json"))
files

In [ ]:
############# Copy from online(not sure if it works or not yet) ##########

#Convert Yelp Academic Dataset from JSON to CSV
import csv
def convert(line):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    data = json.loads(line)
    for k, v in data.items():
        if isinstance(v, list):
            data[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                data['%s_%s' % (k, kk)] = vv
            del data[k]
    return line


csv_filename = 'review.csv'
df = pd.DataFrame([convert(line) for line in file(files[2])])
df.to_csv(csv_filename, index=False)


In [ ]:
def readfile(files):
    # read the file into a python array
    with open(files, 'rb') as f:
        data = f.readlines()
        # remove the trailing "\n" from each line
        data = map(lambda x: x.rstrip(), data)
        data_str = "[" + ','.join(data) + "]"
        df = pd.read_json(data_str)
        
        f.close()
        return df
     
def fixdf(newlist):
    new_df = pd.concat(newlist, axis = 0)
    return new_df

yelplist = []

In [ ]:
review_data = readfile(files[2])

In [ ]:
review_data.head()

### Data Wrangling 

In [12]:
import requests_cache
import requests
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
import numpy as np
import seaborn as sns
from requests_oauthlib import OAuth1Session
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
requests_cache.install_cache("cache")

def yelp_search(term, location):
    """
    makes a search request
    
    Argument: search term, location
    
    Return: search result items as a list (for 0 items, return an empty list)
    """
    consumer_key = "uFKYftN0fu4pD6guW2cYng"
    consumer_secret = "iUYQ2Hk3-vTQVd0zOFnvloWPivA"
    token = "FZnep4FiFmwrrifuNm6M-_K661Wb3J02"
    token_secret = "CRvBo_fO-r4pE_-p2UrnN_mxBlc"
   
    urlbase = "http://api.yelp.com/v2/search"
    params = ({"term": term})
    # 2=Highest Rated 
    params.update({"location": location, "sort": "2", "cll" : "latitude,longitude"})
    session = OAuth1Session(consumer_key, consumer_secret,token,token_secret)
     
    request = session.get(urlbase,params=params)
   
    #Transforms JSON API response into dictionary
    data = request.json()
    session.close()
    output = data
    
    if "businesses" not in output:
        return {}
    
    else:
        return output["businesses"]


In [85]:
def extract(url):
    """
    extract
    """
    info = requests.get(url).content
    wikisoup = BeautifulSoup(info, 'lxml')
    city_data = wikisoup.findAll("th", {"scope": "row"})
    population_data = wikisoup.findAll("td", {"align":"right"})
    city = [city.text for city in city_data]
    city = city[1:-8]
    population_data = wikisoup.findAll("td", {"align":"right"})
    population_data = "\n".join([population.text for population in population_data])
    population_split = population_data.split("\n")
    population = population_split[0::2]
    info_dict = {"city": city, "population": population}
    return info_dict

In [86]:
url = "https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_California"
url_output = extract(url)
pd_scrape = pd.DataFrame(url_output)

In [160]:
# this excel provide latitude and longitude information
pd_location = pd.read_excel("/Users/Stephanie/Desktop/calcities.xlsx", header = 0)

In [161]:
pd_location_1 = pd_location.rename(columns = {'Name':'city'})
pd_geo = pd.merge(pd_scrape, pd_location_1, on = "city")

In [167]:
pd_geo['city'] = pd_geo['city'] + ', CA'

In [60]:
def search(keyterm, city):
    result = yelp_search(keyterm, city)
    df_result = pd.DataFrame(result)
    df_result["city"] = city
    return df_result

In [92]:
keyterm = "restaurants"
yelp_restaurants = [search(keyterm, "%s, CA" %city) for city in pd_scrape["city"]]

In [93]:
df_restaurants = pd.concat(yelp_restaurants, ignore_index = True)

In [96]:
df_food_drop = df_restaurants.drop(["deals", "gift_certificates", "image_url", "location", "menu_date_updated", "menu_provider", "phone", "rating_img_url", "rating_img_url_large", "rating_img_url_small", "snippet_image_url"], 1)
df_food_final = pd.merge(df_food_drop, pd_geo, on = "city", how = "inner")

In [119]:
#change category thats 'Nan' to 'Unknown'
for i in range(0,len(df_food_final)):
    if type(df_food_final['categories'][i]) == float:
        df_food_final['categories'][i] = [[u'Unknown']]

/Users/stephanie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [99]:
keyterm = "hotels"
yelp_hotels = [search(keyterm, "%s, CA" %city) for city in pd_scrape["city"]]
df_hotels = pd.concat(yelp_hotels)

In [100]:
df_hotel_drop = df_hotels.drop(["deals", "gift_certificates", "image_url", "location", "menu_date_updated", "menu_provider", "phone", "rating_img_url", "rating_img_url_large", "rating_img_url_small", "snippet_image_url"], 1)

In [101]:
df_hotels_final = pd.merge(df_hotel_drop, pd_geo, on = "city", how = "inner")

In [235]:
for i in range(0,len(df_hotels_final)):
    if type(df_hotels_final['categories'][i]) == float:
        df_hotels_final['categories'][i] = [[u'Unknown']]

/Users/stephanie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Analying data and plotting:

In [104]:
import plotly
import plotly.plotly as py  
import plotly.tools as tls   
from plotly.graph_objs import *
import numpy as np  
import plotly.graph_objs as go

tls.set_credentials_file(
    username = 'stephanielam',
    api_key = 'fFDGF8NGc2K0WEpiEnSL')

In [256]:
from collections import Counter

#get the most frequent category labels (Overall)
def freq_cat(dataframe):
    count = []
    for i in range(0,len(dataframe)):
        data = dataframe['categories'][i][0][0]
        count.append(data)
        freq = Counter(count)
        
    count_df = pd.DataFrame(freq.most_common()[0:10])
    return count_df

In [259]:
count_df = freq_cat(df_food_final)

#plot
data = [go.Bar(x= count_df[0],y= count_df[1])]
layout = go.Layout(title='Top 10 most frequent category')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'Top 10 most frequent category')

In [203]:
#get star count by each category
def get_count(category):
    count_list =[]
    for i in range(len(df_food_final)):
        if df_food_final['categories'][i][0][0] == category:
            count_list.append(df_food_final['rating'][i])       
    labels, values = zip(*Counter(count_list).items()) 
    count_list =[] 
    return labels, values

cat_count = []
def get_cat_count(dataframe):
    for i in range(0, len(dataframe[0])):
        count = get_count(dataframe[0][i])
        cat_count.append(count)
    
    return cat_count

cat_count = get_cat_count(count_df)

In [202]:
#plot of Distribution of stars(by category)

for i in range(0, len(cat_count)):
    locals()['trace%s' % i] = go.Bar(x = cat_count[i][0], y=cat_count[i][1], name= count_df[0][i])

data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(barmode = 'stack', title ='Rating stars distribution (by categeory)'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Rating stars distribution (by categeory)')

In [302]:
#get the average star rating (by City)
def average_star_city(dataframe, category):
    average_rate = dataframe.groupby('city')[category].mean()
    average_rate = average_rate.reset_index()
    average_rate = pd.DataFrame(average_rate)
    return average_rate
    
rating_star = average_star_city(df_food_final, 'rating')

In [303]:
rating_star = pd.merge(rating_star, pd_geo, on='city')
rating_star

,city,rating,population,Latitude,Longitude
0,"Adelanto, CA",4.425000,"31,765",34.582769,-117.409214
1,"Agoura Hills, CA",4.450000,"20,330",34.153339,-118.761675
2,"Alameda, CA",4.250000,"73,812",37.765206,-122.241636
3,"Albany, CA",4.375000,"18,539",37.886869,-122.297747
4,"Alhambra, CA",4.150000,"83,089",34.095286,-118.127014
5,"Aliso Viejo, CA",4.475000,"47,823",33.575000,-117.725556
6,"Alturas, CA",3.500000,"2,827",41.487114,-120.542456
7,"Amador City, CA",4.275000,185,38.419356,-120.824103
8,"American Canyon, CA",4.275000,"19,454",38.174917,-122.260803
9,"Anaheim, CA",4.475000,"336,265",33.835292,-117.914503


In [198]:
rating_star['text'] = 'City:' + rating_star['city'] + ' \nRating: ' + rating_star['rating'].astype(np.str) +' star'
limits = [(1,3),(3,3.5),(3.5,4),(4,4.5),(4.5,5)]
colors = ["lightgrey","lightgreen","lightblue","lightyellow","red"]
cities = []
scale = 10

for i in range(len(limits)):
    lim = limits[i]
    rate_sub = rating_star.query('%f' %lim[0] +'< rating <= %f' %lim[1])
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = rate_sub['Longitude'],
        lat = rate_sub['Latitude '],
        text = rate_sub['text'],
        marker = dict(
            size = rate_sub['rating'] * scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]))
    cities.append(city)

layout = dict(
        title = 'Restaurant Average Rating by City<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot(fig, validate=False, filename='d3-bubble-map' )

In [356]:
#rating vs review_count
data = go.Scatter(
    x = df_food_final['rating'],
    y = df_food_final['review_count'],
    mode = 'markers'
)

layout = go.Layout(
    title='Scatter plot of Restaurant\'s rating vs review count',
    xaxis=dict(
        title='Rating',
        gridcolor='rgb(255, 255, 255)',
    ),
    yaxis=dict(
        title='Review Count',
        gridcolor='rgb(255, 255, 255)',
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = dict(data=[data], layout=layout)
py.iplot(fig, filename='Scatter plot of rating vs review count')

In [385]:
#top 5 restaurants:
best_5 = df_food_final.groupby('rating')['review_count'].nlargest(5).reset_index()
best_5 = pd.DataFrame(best_5)

In [394]:
df_best_5 = pd.DataFrame()
for i in range(0,len(best_5)):
    if best_5['rating'][i] == 5.0:
        data = df_food_final.iloc[[best_5['level_1'][i]]]
        df_best_5 = pd.concat([data,df_best_5],ignore_index = True)
        
df_best_5

,categories,city,display_phone,id,is_claimed,is_closed,mobile_url,name,rating,review_count,snippet_text,url,population,Latitude,Longitude
0,"[[American (New), newamerican], [Farmers Marke...","Lompoc, CA",+1-805-344-4609,bell-street-farm-los-alamos,True,False,https://m.yelp.com/biz/bell-street-farm-los-al...,Bell Street Farm,5.0,437,I stopped by for a quick snack in between meal...,https://www.yelp.com/biz/bell-street-farm-los-...,"42,434",34.639150,-120.457942
1,"[[Coffee & Tea, coffee], [Sandwiches, sandwich...","Burlingame, CA",+1-650-343-1110,jougert-bar-burlingame,True,False,https://m.yelp.com/biz/jougert-bar-burlingame?...,Jougert Bar,5.0,455,You cannot go wrong when you come to this plac...,https://www.yelp.com/biz/jougert-bar-burlingam...,"28,806",37.584103,-122.366083
2,"[[Delis, delis], [Bakeries, bakeries], [Sandwi...","La Quinta, CA",+1-760-347-7770,tkb-bakery-and-deli-indio-2,True,False,https://m.yelp.com/biz/tkb-bakery-and-deli-ind...,TKB Bakery & Deli,5.0,1052,I have a hard time choosing between \nThe Iron...,https://www.yelp.com/biz/tkb-bakery-and-deli-i...,"37,467",33.646692,-116.310008
3,"[[Delis, delis], [Bakeries, bakeries], [Sandwi...","Indio, CA",+1-760-347-7770,tkb-bakery-and-deli-indio-2,True,False,https://m.yelp.com/biz/tkb-bakery-and-deli-ind...,TKB Bakery & Deli,5.0,1052,I have a hard time choosing between \nThe Iron...,https://www.yelp.com/biz/tkb-bakery-and-deli-i...,"76,036",33.720578,-116.215561
4,"[[Delis, delis], [Bakeries, bakeries], [Sandwi...","Coachella, CA",+1-760-347-7770,tkb-bakery-and-deli-indio-2,True,False,https://m.yelp.com/biz/tkb-bakery-and-deli-ind...,TKB Bakery & Deli,5.0,1052,I have a hard time choosing between \nThe Iron...,https://www.yelp.com/biz/tkb-bakery-and-deli-i...,"40,704",33.680300,-116.173894


### Hotels

In [358]:
#get the most frequent category labels (Overall)
count_df = freq_cat(df_hotels_final)

In [360]:
#plot
data = [go.Bar(x= count_df[0],y= count_df[1])]
layout = go.Layout(title='Top 10 most frequent category')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'Top 10 most frequent category')

In [281]:
#restrict to only the category 'Hotels'
df_limit_hotels = pd.DataFrame()
for i in range(0, len(df_hotels_final)):
    for j in range(0, len(df_hotels_final['categories'][i])):
        if df_hotels_final['categories'][i][j] == [u'Hotels', u'hotels']:
            df_limit_hotels = pd.concat([df_hotels_final.iloc[[i]],limit_hotels], ignore_index = True)

In [336]:
#get the average star rating (by City)
hotel_rating = average_star_city(df_limit_hotels, 'rating')
average_review = average_star_city(df_limit_hotels, 'review_count')

In [337]:
hotel_rating = pd.merge(hotel_rating, average_review, on= 'city')
hotel_rating = pd.merge(hotel_rating, pd_geo, on='city')

In [335]:
hotel_rating['text'] = 'City:' + hotel_rating['city'] + '<br>Rating: ' + hotel_rating['rating'].astype(np.str)+' star' + '<br>Average Reviews count: ' + hotel_rating['review_count'].astype(np.str)
limits = [(1,3),(3,3.5),(3.5,4),(4,4.5),(4.5,5)]
colors = ["lightgrey","lightgreen","lightblue","lightyellow","red"]
cities = []
scale = 10

for i in range(len(limits)):
    lim = limits[i]
    rate_sub = hotel_rating.query('%f' %lim[0] +'< rating <= %f' %lim[1])
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = rate_sub['Longitude'],
        lat = rate_sub['Latitude '],
        text = rate_sub['text'],
        marker = dict(
            size = rate_sub['rating'] * scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]))
    cities.append(city)

layout = dict(
        title = 'Hotel Average Rating by City<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot(fig, validate=False, filename='hotel average rating' )

In [355]:
#rating vs review_count
trace = go.Scatter(
    x = df_hotels_final['rating'],
    y = df_hotels_final['review_count'],
    mode = 'markers'
)

layout = go.Layout(
    title='Scatter plot of Hotel\'s rating vs review count',
    xaxis=dict(
        title='Rating',
        gridcolor='rgb(255, 255, 255)',
    ),
    yaxis=dict(
        title='Review Count',
        gridcolor='rgb(255, 255, 255)',
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = dict(data=[data], layout=layout)
py.iplot(fig, filename='Scatter plot of rating vs review count2')

In [399]:
#Top five hotels:
best_10 = df_limit_hotels.groupby('rating')['review_count'].nlargest(10).reset_index()
best_10 = pd.DataFrame(best_5)

df_best_10 = pd.DataFrame()
for i in range(0,len(best_5)):
    if best_10['rating'][i] == 5.0:
        data = df_limit_hotels.iloc[[best_10['level_1'][i]]]
        df_best_10 = pd.concat([data,df_best_10],ignore_index = True)
        
df_best_10

,categories,city,display_phone,id,is_claimed,is_closed,mobile_url,name,rating,review_count,snippet_text,url,population,Latitude,Longitude
0,"[[Hotels, hotels]]","South Lake Tahoe, CA",+1-530-544-5515,fireside-lodge-south-lake-tahoe,True,False,https://m.yelp.com/biz/fireside-lodge-south-la...,Fireside Lodge,5.0,123,We had the privilege of staying here again. An...,https://www.yelp.com/biz/fireside-lodge-south-...,"21,403",38.939925,-119.977186
1,"[[Hotels, hotels], [Venues & Event Spaces, ven...","Cathedral City, CA",+1-760-325-7100,the-rossi-hotel-palm-springs,True,False,https://m.yelp.com/biz/the-rossi-hotel-palm-sp...,The Rossi Hotel,5.0,127,I decided to spend 3 nights and 4 days here af...,https://www.yelp.com/biz/the-rossi-hotel-palm-...,"51,200",33.779742,-116.465292
2,"[[Hotels, hotels], [Venues & Event Spaces, ven...","Palm Springs, CA",+1-760-325-7100,the-rossi-hotel-palm-springs,True,False,https://m.yelp.com/biz/the-rossi-hotel-palm-sp...,The Rossi Hotel,5.0,127,I decided to spend 3 nights and 4 days here af...,https://www.yelp.com/biz/the-rossi-hotel-palm-...,"44,552",33.830297,-116.545292
3,"[[Hotels, hotels]]","Cathedral City, CA",+1-760-320-1178,palm-springs-rendezvous-palm-springs,True,False,https://m.yelp.com/biz/palm-springs-rendezvous...,Palm Springs Rendezvous,5.0,188,This is an incredibly special boutique B & B! ...,https://www.yelp.com/biz/palm-springs-rendezvo...,"51,200",33.779742,-116.465292
4,"[[Hotels, hotels]]","Desert Hot Springs, CA",+1-760-320-1178,palm-springs-rendezvous-palm-springs,True,False,https://m.yelp.com/biz/palm-springs-rendezvous...,Palm Springs Rendezvous,5.0,188,This is an incredibly special boutique B & B! ...,https://www.yelp.com/biz/palm-springs-rendezvo...,"25,938",33.961125,-116.501678
5,"[[Hotels, hotels]]","Palm Springs, CA",+1-760-320-1178,palm-springs-rendezvous-palm-springs,True,False,https://m.yelp.com/biz/palm-springs-rendezvous...,Palm Springs Rendezvous,5.0,188,This is an incredibly special boutique B & B! ...,https://www.yelp.com/biz/palm-springs-rendezvo...,"44,552",33.830297,-116.545292
6,"[[Hotels, hotels]]","Rancho Mirage, CA",+1-760-320-1178,palm-springs-rendezvous-palm-springs,True,False,https://m.yelp.com/biz/palm-springs-rendezvous...,Palm Springs Rendezvous,5.0,188,This is an incredibly special boutique B & B! ...,https://www.yelp.com/biz/palm-springs-rendezvo...,"17,218",33.739744,-116.412789
7,"[[Hotels, hotels], [Bed & Breakfast, bedbreakf...","Fairfield, CA",+1-707-253-1331,the-inn-on-first-napa,True,False,https://m.yelp.com/biz/the-inn-on-first-napa?a...,The Inn On First,5.0,340,"My boyfriend and I recently stayed one rainy, ...",https://www.yelp.com/biz/the-inn-on-first-napa...,"105,321",38.249358,-122.039967
8,"[[Hotels, hotels], [Bed & Breakfast, bedbreakf...","Napa, CA",+1-707-253-1331,the-inn-on-first-napa,True,False,https://m.yelp.com/biz/the-inn-on-first-napa?a...,The Inn On First,5.0,340,"My boyfriend and I recently stayed one rainy, ...",https://www.yelp.com/biz/the-inn-on-first-napa...,"76,915",38.297539,-122.286864
9,"[[Hotels, hotels], [Bed & Breakfast, bedbreakf...","Sonoma, CA",+1-707-253-1331,the-inn-on-first-napa,True,False,https://m.yelp.com/biz/the-inn-on-first-napa?a...,The Inn On First,5.0,340,"My boyfriend and I recently stayed one rainy, ...",https://www.yelp.com/biz/the-inn-on-first-napa...,"10,648",38.291858,-122.458036
